### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit
%cd ..

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/to-run-for-results/cicddos2019-topfeat/baselines
Obtaining file:///project/to-run-for-results/cicddos2019-topfeat/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
Requirement already up-to-date: tensorflow-gpu==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
/project/to-run-for-results/cicddos2019-topfeat


In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9427023311966841678,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 574503465540895610
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12997927059606995142
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
cicddos2019 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
data = pd.concat([cicddos2019], ignore_index=True)
print(len(data.columns))
data.head(5)


67


,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,1472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
1,1,2,0,2896.0,0.0,1448.0,1448.0,1448.0,0.0,0.0,...,1480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
2,1,2,0,2526.0,0.0,1263.0,1263.0,1263.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
3,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
4,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS


In [4]:
cicdos2017_features = ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"]
cicids2017_features = ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"]
cicddos2019_features = ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"]
nslkdd_features = ["dst_host_serror_rate", "service_private", "count","dst_host_count", "service_domain_u", "flag_REJ", "dst_host_diff_srv_rate"]
unsw_features = ["sttl", "dttl", "ct_state_ttl", "service", "dload", "rate", "dmean", "dbytes", "dur", "is_sm_ips_ports", "dloss"]

important_features = cicddos2019_features + ["Label"] # Adding class for custom environment logic
important_features = list(set(important_features))
print(len(important_features))

removable_features = data.columns ^ important_features
print(removable_features.shape)
data = data.drop(labels=removable_features, axis='columns')

6
(61,)


In [5]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Fwd Packets Length Total  138007 non-null  float32 
 1   Bwd Packet Length Min     138007 non-null  float32 
 2   ACK Flag Count            138007 non-null  int8    
 3   URG Flag Count            138007 non-null  int8    
 4   Down/Up Ratio             138007 non-null  float32 
 5   Label                     138007 non-null  category
dtypes: category(1), float32(3), int8(2)
memory usage: 2.0 MB


,Fwd Packets Length Total,Bwd Packet Length Min,ACK Flag Count,URG Flag Count,Down/Up Ratio
count,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000
mean,4.018699e+03,25.685892,0.198584,0.275283,0.702986
std,5.141442e+04,52.278210,0.398936,0.446658,0.937631
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.800000e+01,0.000000,0.000000,0.000000,0.000000
50%,8.600000e+01,0.000000,0.000000,0.000000,1.000000
75%,2.064000e+03,46.000000,0.000000,1.000000,1.000000
max,1.526642e+07,1460.000000,1.000000,1.000000,23.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [6]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [7]:
try:
    data.drop('Timestamp', inplace=True, axis=1)
except:
    pass
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)

Benign           95092
DrDoS_NTP        13687
TFTP             13032
Syn               6734
UDP               2031
MSSQL             1338
DrDoS_MSSQL       1325
DrDoS_UDP         1232
UDP-lag           1083
DrDoS_DNS          682
DrDoS_SSDP         473
DrDoS_SNMP         410
LDAP               330
DrDoS_LDAP         210
DrDoS_NetBIOS      144
Portmap            122
NetBIOS             70
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [8]:
x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

Label
Index(['Fwd Packets Length Total', 'Bwd Packet Length Min', 'ACK Flag Count',
       'URG Flag Count', 'Down/Up Ratio'],
      dtype='object')
0.0    95092
1.0    42915
Name: Label, dtype: int64
42915
95092
52177
Int64Index([104602,  68073, 132797,  64149,  95024,  37389,  82519,  27264,
             73445, 118594,
            ...
            109217, 109270,  81581,  81947,  94497, 105757,  57297,  56845,
            136018, 116800],
           dtype='int64', length=52177)
0.0    42915
1.0    42915
Name: Label, dtype: int64
38623


In [9]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()

# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


### Custom environment

In [10]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(5,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [11]:
def ids_ddqn():
    env = IdsEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        policy_kwargs=dict(dueling=True),
        double_q=True,
        verbose=1, 
        learning_rate=0.00025,
        buffer_size=1000000,
        exploration_fraction=0.1,
        exploration_final_eps=0.1,
        train_freq=4,
        learning_starts=50000,
        target_network_update_freq=10000,
        gamma=0.99,
        param_noise=True,
        prioritized_replay=False,
        prioritized_replay_alpha=0.6,
        batch_size=32,
    )
    model.learn(
        total_timesteps=int(1.0e5),
        log_interval=10000,
        
    )

    env.close()
    
    return model

start_time = time.time()
ddqn_model_1 = ids_ddqn()
print()
print("DQN 1 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_2 = ids_ddqn()
print()
print("DQN 2 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_3 = ids_ddqn()
print()
print("DQN 3 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_4 = ids_ddqn()
print()
print("DQN 4 Training Time:", time.time() - start_time)








Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| % time spent exploring  | 10       |
| episodes                | 10000    |
| mean 100 episode reward | 0.4      |
| steps                   | 9999     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 20000    |
| mean 100 episode reward | 0.5      |
| steps                   | 19999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 30000    |
| mean 100 episode reward | 0.5      |
| steps                   | 29999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 40000    |
| mean 100 episode reward | 0.5      |
| steps                   | 39999    |
-------

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 10       |
| episodes                | 10000    |
| mean 100 episode reward | 0.3      |
| steps                   | 9999     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 20000    |
| mean 100 episode reward | 0.5      |
| steps                   | 19999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 30000    |
| mean 100 episode reward | 0.5      |
| steps                   | 29999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 40000    |
| mean 100 episode reward | 0.4      |
| steps                   | 39999    |
--------------------------------------
--------------------------------------
| % time spent exploring 

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 10       |
| episodes                | 10000    |
| mean 100 episode reward | 0.5      |
| steps                   | 9999     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 20000    |
| mean 100 episode reward | 0.5      |
| steps                   | 19999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 30000    |
| mean 100 episode reward | 0.5      |
| steps                   | 29999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 40000    |
| mean 100 episode reward | 0.5      |
| steps                   | 39999    |
--------------------------------------
--------------------------------------
| % time spent exploring 

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 10       |
| episodes                | 10000    |
| mean 100 episode reward | 0.4      |
| steps                   | 9999     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 20000    |
| mean 100 episode reward | 0.5      |
| steps                   | 19999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 30000    |
| mean 100 episode reward | 0.5      |
| steps                   | 29999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 40000    |
| mean 100 episode reward | 0.5      |
| steps                   | 39999    |
--------------------------------------
--------------------------------------
| % time spent exploring 

In [12]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        try:
            recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        except:
            recall = 0
        try:
            FPR = (float(FP) / (TN + FP)) # 1 - specificity
        except:
            FPR = 0
        try:
            f1_score = 2 * (precision * recall) / (precision + recall)
        except:
            f1_score = 0
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]

models = [ddqn_model_1, ddqn_model_2, ddqn_model_3, ddqn_model_4]

results_1 = ids_eval(ddqn_model_1)
results_2 = ids_eval(ddqn_model_2)
results_3 = ids_eval(ddqn_model_3)
results_4 = ids_eval(ddqn_model_4)
total_results = [-1,-1,-1,-1,-1]
accuracies = [results_1[0], results_2[0], results_3[0], results_4[0]]

for i in range(len(results_1)):
    total_results[i] = (results_1[i] + results_2[i] + results_3[i] + results_4[i] )/ 4

print()    
print('Total validation done...')
print('Accuracy: {0}%'.format(total_results[0] * 100))
print('Precision: {0}%'.format(total_results[1] * 100))
print('Recall/TPR/Sensitivity: {0}%'.format(total_results[2] * 100))
print('FPR: {0}%'.format(total_results[3] * 100))
print('F1 score: {0}'.format(total_results[4]))

highest = 0
for i in range(4):
    if accuracies[i] > accuracies[highest]:
        highest = i

print()
print("Saving model {0}".format(highest + 1))
models[highest].save('pn_dddqn_cicddos2019_allfeat.pkl')



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 57.389277389277396%
Precision: 99.20745920745921%
Recall/TPR/Sensitivity: 54.02386392485402%
FPR: 4.843304843304843%
F1 score: 0.6995397764628534

validation done...
Accuracy: 85.82750582750582%
Precision: 71.84149184149184%
Recall/TPR/Sensitivity: 99.7411003236246%
FPR: 22.003642987249545%
F1 score: 0.835230352303523

validation done...
Accuracy: 50.0%
Precision: 100.0%
Recall/TPR/Sensitivity: 50.0%
FPR: 0%
F1 score: 0.6666666666666666

validation done...
Accuracy: 50.18648018648019%
Precision: 99.95337995337995%
Recall/TPR/Sensitivity: 50.09345794392524%
FPR: 10.0%
F1 score: 0.6673929961089495

Total validation done...
Accuracy: 60.850815850815856%
Precision: 92.75058275058275%
Recall/TPR/Sensitivity: 63.46460554810096%
FPR: 9.211736957638598%
F1 score: 0.7172074478854982

Saving model 2
